In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import os

(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train_mlp = x_train
x_test_mlp = x_test

x_train_cnn = np.expand_dims(x_train, -1)
x_test_cnn = np.expand_dims(x_test, -1)

print("MLP input shape:", x_train_mlp.shape)
print("CNN input shape:", x_train_cnn.shape)

2025-11-10 11:51:54.306411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762775514.475233      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762775514.520795      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
MLP input shape: (60000, 28, 28)
CNN input shape: (60000, 28, 28, 1)


In [2]:
mlp_model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

mlp_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("MLP Model Summary:")
mlp_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1762775529.074963      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


MLP Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 235,146 (918.54 KB)

 Trainable params: 235,146 (918.54 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
cnn_model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("CNN Model Summary:")
cnn_model.summary()

CNN Model Summary:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,714 (221.54 KB)

 Trainable params: 56,714 (221.54 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
history_mlp = mlp_model.fit(
    x_train_mlp, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=64,
    verbose=2
)

history_cnn = cnn_model.fit(
    x_train_cnn, y_train,
    validation_split=0.1,
    epochs=5,
    batch_size=64,
    verbose=2
)

Epoch 1/5


I0000 00:00:1762775532.138781      59 service.cc:148] XLA service 0x79310c004f50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762775532.139131      59 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1762775532.332934      59 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1762775532.977262      59 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


844/844 - 5s - 6ms/step - accuracy: 0.8225 - loss: 0.4981 - val_accuracy: 0.8442 - val_loss: 0.4245
Epoch 2/5
844/844 - 2s - 2ms/step - accuracy: 0.8660 - loss: 0.3666 - val_accuracy: 0.8577 - val_loss: 0.3793
Epoch 3/5
844/844 - 2s - 2ms/step - accuracy: 0.8797 - loss: 0.3267 - val_accuracy: 0.8757 - val_loss: 0.3429
Epoch 4/5
844/844 - 2s - 2ms/step - accuracy: 0.8878 - loss: 0.3054 - val_accuracy: 0.8802 - val_loss: 0.3271
Epoch 5/5
844/844 - 2s - 2ms/step - accuracy: 0.8941 - loss: 0.2868 - val_accuracy: 0.8788 - val_loss: 0.3286
Epoch 1/5
844/844 - 7s - 9ms/step - accuracy: 0.7930 - loss: 0.5773 - val_accuracy: 0.8538 - val_loss: 0.4134
Epoch 2/5
844/844 - 2s - 3ms/step - accuracy: 0.8631 - loss: 0.3844 - val_accuracy: 0.8733 - val_loss: 0.3627
Epoch 3/5
844/844 - 2s - 3ms/step - accuracy: 0.8781 - loss: 0.3390 - val_accuracy: 0.8815 - val_loss: 0.3232
Epoch 4/5
844/844 - 2s - 3ms/step - accuracy: 0.8886 - loss: 0.3085 - val_accuracy: 0.8805 - val_loss: 0.3276
Epoch 5/5
844/844 - 

In [5]:
mlp_test_loss, mlp_test_acc = mlp_model.evaluate(x_test_mlp, y_test, verbose=0)
cnn_test_loss, cnn_test_acc = cnn_model.evaluate(x_test_cnn, y_test, verbose=0)

print(f"MLP Test Accuracy: {mlp_test_acc:.4f}, Loss: {mlp_test_loss:.4f}")
print(f"CNN Test Accuracy: {cnn_test_acc:.4f}, Loss: {cnn_test_loss:.4f}")

MLP Test Accuracy: 0.8729, Loss: 0.3527
CNN Test Accuracy: 0.8862, Loss: 0.3223


In [6]:
mlp_params = mlp_model.count_params()
cnn_params = cnn_model.count_params()

print(f"Trainable Parameters — MLP: {mlp_params:,}")
print(f"Trainable Parameters — CNN: {cnn_params:,}")

mlp_model.save("mlp_model.h5")
cnn_model.save("cnn_model.h5")

mlp_size_mb = os.path.getsize("mlp_model.h5") / (1024 * 1024)
cnn_size_mb = os.path.getsize("cnn_model.h5") / (1024 * 1024)

print(f"MLP Model Size: {mlp_size_mb:.2f} MB")
print(f"CNN Model Size: {cnn_size_mb:.2f} MB")

Trainable Parameters — MLP: 235,146
Trainable Parameters — CNN: 56,714
MLP Model Size: 2.72 MB
CNN Model Size: 0.69 MB


In [7]:
def estimate_flops_mlp(input_dim, hidden_dims, output_dim):
    flops = 0
    prev_dim = input_dim
    for h in hidden_dims:
        flops += 2 * prev_dim * h
        prev_dim = h
    flops += 2 * prev_dim * output_dim
    return flops

def estimate_flops_cnn():
    flops = (26*26*16*3*3) + (13*13*32*3*3*16) + (1600*64) + (64*10)
    return 2 * flops

flops_mlp_inf = estimate_flops_mlp(784, [256, 128], 10)
flops_cnn_inf = estimate_flops_cnn()

flops_mlp_train = 3 * flops_mlp_inf
flops_cnn_train = 3 * flops_cnn_inf

mem_mlp_train_mb = mlp_params * 4 * 3 / (1024**2)
mem_cnn_train_mb = cnn_params * 4 * 3 / (1024**2)

print(f"MLP Inference FLOPs ≈ {flops_mlp_inf/1e6:.2f} MFLOPs")
print(f"CNN Inference FLOPs ≈ {flops_cnn_inf/1e6:.2f} MFLOPs")
print(f"MLP Training Memory ≈ {mem_mlp_train_mb:.2f} MB")
print(f"CNN Training Memory ≈ {mem_cnn_train_mb:.2f} MB")

MLP Inference FLOPs ≈ 0.47 MFLOPs
CNN Inference FLOPs ≈ 1.96 MFLOPs
MLP Training Memory ≈ 2.69 MB
CNN Training Memory ≈ 0.65 MB


In [8]:
import pandas as pd

results = pd.DataFrame({
    "Model": ["MLP", "CNN"],
    "Test Accuracy": [mlp_test_acc, cnn_test_acc],
    "Trainable Parameters": [mlp_params, cnn_params],
    "Saved Model Size (MB)": [mlp_size_mb, cnn_size_mb],
    "FLOPs (Training)": [f"{flops_mlp_train/1e6:.1f}M", f"{flops_cnn_train/1e6:.1f}M"],
    "FLOPs (Inference)": [f"{flops_mlp_inf/1e6:.1f}M", f"{flops_cnn_inf/1e6:.1f}M"],
    "Training Memory (MB)": [f"{mem_mlp_train_mb:.1f}", f"{mem_cnn_train_mb:.1f}"]
})

display(results)

,Model,Test Accuracy,Trainable Parameters,Saved Model Size (MB),FLOPs (Training),FLOPs (Inference),Training Memory (MB)
0,MLP,0.8729,235146,2.721703,1.4M,0.5M,2.7
1,CNN,0.8862,56714,0.687439,5.9M,2.0M,0.6


In [9]:
mlp_model = tf.keras.models.load_model("mlp_model.h5")
cnn_model = tf.keras.models.load_model("cnn_model.h5")

In [10]:
(x_train, y_train), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype("float32") / 255.0

In [11]:
def representative_dataset_gen_mlp():
    for i in range(100):
        img = x_train[i]
        img = np.expand_dims(img, axis=0).astype(np.float32)
        yield [img]

def representative_dataset_gen_cnn():
    for i in range(100):
        img = x_train[i]
        img = np.expand_dims(img, axis=(0, -1)).astype(np.float32)  # (1,28,28,1)
        yield [img]

def convert_and_save(model, rep_gen, out_path, input_type=tf.int8, output_type=tf.int8):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = rep_gen
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = input_type
    converter.inference_output_type = output_type

    tflite_model = converter.convert()
    with open(out_path, "wb") as f:
        f.write(tflite_model)
    size_mb = os.path.getsize(out_path) / (1024 * 1024)
    print(f"Saved {out_path} ({size_mb:.3f} MB)")
    return out_path, size_mb

In [12]:
convert_and_save(mlp_model, representative_dataset_gen_mlp, "mlp_model_int8.tflite")

Saved artifact at '/tmp/tmp44eqc_63'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  133255638133136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133255638134672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133255638131024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133255638132368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133255638134864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133254546865808: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1762775565.178467      19 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1762775565.178503      19 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1762775565.183351      19 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8


Saved mlp_model_int8.tflite (0.237 MB)


('mlp_model_int8.tflite', 0.23702239990234375)

In [13]:
convert_and_save(cnn_model, representative_dataset_gen_cnn, "cnn_model_int8.tflite")

Saved artifact at '/tmp/tmpradycuqw'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  133254184388368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133254184390480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133254184389520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133254184390672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133254184390864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133254184391632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133254184391248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133254184392400: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1762775566.462374      19 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1762775566.462412      19 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8


Saved cnn_model_int8.tflite (0.061 MB)


('cnn_model_int8.tflite', 0.0609588623046875)